# Предсказание цен Яндекс Недвижимости

**Цель**: Улучшение коммуникации между арендодателями/продавцами и потенциальными арендаторами/покупателями путем предоставления объективной внешней оценки стоимости квартиры.

**Задача**: Разработка MVP алгоритма на основе машинного обучения для оценки рыночной стоимости квартиры по её характеристикам.

**Заказчик**: Продакт-менеджмент Яндекс Недвижимость.

**Исходные данные**: Данные о квартирах и домах из сервиса Яндекс Недвижимость., собранные посредством Airflow в таблице `flats_fstore` на сервере.

In [ ]:
# Импорт библиотек, классов и функций
import os
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import sweetviz as sv
from IPython.display import display
from sqlalchemy import create_engine
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
# Определение констант и настройка
RANDOM_STATE = 42 # Ответ на главный вопрос жизни, вселенной и всего
TEST_SIZE = .25
TBL = 'flats_fstore' # хранилище признаков без очистки и преобразований

# Креды для подключения к БД
PG_HOST = os.environ.get("PG_HOST")
PG_PORT = os.environ.get("PG_PORT")
PG_USER = os.environ.get("PG_USER")
PG_PASS = os.environ.get("PG_PASS")
PG_DB = os.environ.get("PG_DB")
CON_STR = f'postgresql://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}'

## Загрузка данных

In [ ]:
# Функция подключения к базе данных и получение датасета
def get_data(con_str=CON_STR, tbl=TBL):
    con = create_engine(con_str)
    result = pd.read_sql(f"select * from {tbl}", con)
    con.dispose()
    return result 

In [ ]:
# Загрузка данных
data = get_data()
data.info()

Хранилище признаков содержит 141'362 записи, пропуски отсутствуют. Таблица состоит из следующих 17 полей:
- `id` — ID квартиры,
- `rooms` — количество комнат,
- `total_area` — общая площадь квартиры,
- `kitchen_area` — площадь кухни,
- `living_area` — площадь гостиной,
- `floor` — этаж, на котором находится квартира,
- `studio` — является ли квартира студией,
- `is_apartment` — является ли квартира апартаментами,
- `building_type_int` — тип здания,
- `build_year` — год постройки,
- `latitude` — широта, на которой находится дом,
- `longitude` — долгота, на которой находится дом,
- `ceiling_height` — высота потолков в здании,
- `flats_count` — общее количество квартир,
- `floors_total` — общее количество этажей,
- `has_elevator` — наличие лифта.
- `price` — цена квартиры,

Признак признак `price` является целевым, а признак `id` не будет использоваться для обучения модели и будет удален. Перейдем к обзору датасета.

## Общий обзор датасета

In [ ]:
# Вывод отчета SweetVIZ
report_floor = sv.analyze(data)
report_floor.show_notebook()

По признакам датасета можно отметить следующее:
1. Из 17 признаков предположительно 4 являются категориальными (`is_apartment`, `studio`, `building_type_int` и `has_elevator`), а остальные числовыми.
2. По признаку `studio` имеется только значение `False`. Возможно в датасет не выгружаются квартиры-студии, соответственно данный признак будет лишним для обучения модели. По признаку `is_apartment` основная масса записей (более 99%) имеет значение `False`, также порядка 90% всех квартир имеют в доме лифт.
3. Имеются большие выбросы в признаках площади (`kitchen_area`, `living_area`, `total_area`), а также в признаках `rooms`, `ceiling_height` и целевом признаке `price`.
4. Достаточно большое количество нулевых значений в признаках `kitchen_area` (8%) и `living_area` (13%).

По целевому признаку наибольшая корреляция наблюдается с числовыми признаками `total_area` (0.42), `rooms` (0.26), `ceiling_height` (0.22) и `living_area` (0.18), а также с категориальной `building_type_int`. Оценим признаки датасета.

### Дубликаты в данных

Оценим количество явных дубликатов в датасете.

In [ ]:
feature_cols = data.drop(columns=["id"]).columns.to_list()
print("Количество неявных дубликатов:",
      data.duplicated(subset=feature_cols).sum())

Вероятнее всего это одни и те же квартиры, так как объявления на продажу могут подаваться как ее хозяином, так и посредником (и возможно не одним). Также могут продаваться однотипные квартиры по одному адресу, например в новостройках. Так как дубликаты не несут дополнительной информации для моделей машинного обучения, избавимся от дублирующихся записей.

In [ ]:
data = data.drop_duplicates(subset=feature_cols).reset_index(drop=True)
print("Размер датасета после удаления дубликатов:", data.shape[0])
del feature_cols

## Анализ признаков для модели

### Количество комнат

По количеству комнат значения начинаются с 1, а максимальное значение доходит до 20-ти. Основная масса квартир до 4-х комнат, это порядка 95% от датасета, а на квартиры более 6-ти комнат приходится менее 0,2%. Оценим распределение комнат по цене в разрезе типов зданий. Так как имеются большие выбросы по целевому признаку, график построим с логарифмической шкалой.

In [ ]:
fig = px.strip(
    data, x="rooms", y="price", color="building_type_int", log_y=True,
    title="Диаграмма рассеяния количества комнат к цене по типам зданий",
    labels={"price": "Цена", "rooms": "Количество комнат", "building_type_int": "Тип здания"},
    category_orders={"building_type_int": range(7)}
)
fig.show(renderer="png", width=1400)

Заметен рост стоимости квартир по мере увеличения количества комнат, однако рост нелинейный. Так как данная шкала логарифмическая, вероятнее всего имеется степенная зависимость между количеством комнат и стоимостью квартиры. Также по типу здания заметно, что более высокие значения типа здания представлены преимущественно для квартир с меньшим количеством комнат и по мере их роста увеличивается преобладание меньших значений типа здания. Можно сделать предположение, что численные типы зданий могут означать различные классы зданий - от элитного до эконом, однако могут иметь и соврешнно иную основу.

Дополнительно заметна группа квартир, стоимость которых существенно ниже основной массы, количество комнат в основном от 1 до 3, но таких объявлений немного.

Чтобы избавиться от выбросов - возможных некорректных и/или редких значений количества комнат, ограничим значения данного признака в пределах от 1 до значений не превышающих 1.5-кратный межквартильный размах свыше 3-го квартиля. Все значения, выходящие за пределы этого промежутка подлежат удалению. Подготовим фукнцию, которая будет определять, является или нет значение выбросом.

In [ ]:
# Функция для поиска выбросов
def is_outlier(
    df: pd.DataFrame, target: str, min_val: int | None = None, 
    max_val: int | None = None, threshold=1.5
) -> pd.Series:
    Q1 = df[target].quantile(0.25)
    Q3 = df[target].quantile(0.75)
    min_val = max(Q1 - threshold * (Q3 - Q1), min_val) if min_val else Q1 - threshold * (Q3 - Q1)
    print("Минимальное значение ниже которого выбросы:", min_val)
    max_val = min(threshold * (Q3 - Q1) + Q3, max_val) if max_val else Q3 + threshold * (Q3 - Q1)
    print("Максимальное значение выше которого выбросы:", max_val)
    return (df[target] < min_val) | (df[target] > max_val)

In [ ]:
print("Количество выбросов признака rooms:", is_outlier(data, "rooms", 1).sum())

Исключим выбросы по признаку rooms:

In [ ]:
data = data[~is_outlier(data, "rooms", min_val=1)]
print("Количество записей после удаления выбросов:", data.shape[0])

### Площадь

В датасете представлены 3 вида площади:
- Общая площадь `total_area`;
- Площадь кухни `kitchen_area`;
- Жилая площадь `living_area`.

**Общая площадь** имеет большой разброс значений в пределах от 11 до более 960 квадратных метров. Наибольшая корреляция с признаком количества комнат. Также имеется корреляция с другими площадями (жилая / кузня) и высотой потолков, а в части категориальных - с типом здания. Оценим распределение количества комнат к общей площади в разрезе типов здания.

In [ ]:
fig = px.strip(
    data, x="rooms", y="total_area", color="building_type_int", 
    title="Диаграмма рассеяния количества комнат к цене по типам зданий",
    labels={"total_area": "Общая площадь", "rooms": "Количество комнат", "building_type_int": "Тип здания"},
    category_orders={"building_type_int": range(7)}
)
fig.show(renderer="png", width=1400)

По графику заметно, что по мере увеличения количества комнат увеличивается и общая площадь. Также для более распространенных типов зданий заметно ранжирование внутри каждой группы по количеству комнат, а также изменение соотношения одних типов зданий с другими по мере увеличения количества комнат. Наиболее приемлемым вариантом в таком случае будет отсеивать выбросы в разрезе количества комнат и типов зданий.

Оценим распределение крайних значений для выбросов на основе межквартильного интервала.

In [ ]:
data.groupby(
    ["rooms", "building_type_int"]
)["total_area"].quantile(
    [0, .01, .05, .25, .5, .75, .95, .99, 1]
).to_frame().unstack().droplevel(0, axis=1).apply(
    lambda x: pd.Series({
        "min": x[0.00], "min_out": x[0.25] - 1.5 * (x[0.75] - x[0.25]), "1%": x[0.01], 
        "5%": x[0.05], "Q1": x[0.25], "Q2": x[0.50], "Q3":x[0.75], "95%": x[0.95], 
        "99%": x[0.99], "max_out": x[0.75] + 1.5 * (x[0.75] - x[0.25]), "max": x[1.00]
}), axis=1).style.background_gradient(
    cmap=sns.diverging_palette(250, 25, as_cmap=True), axis=1
).format("{:,.2f}")

Предоставленные данные имеют достаточно сложное распределение, для некоторых значений площади межквартильный размах явно занижает максимальные значения для определения выброса, либо сильно превосходит имеющиеся значения. Чаще всего пользователи достаточно ответственно подходят к указанию общей площади, однако лучше отсекать сильно отличающиеся значения, на случай ошибок в заполнении. 

Доработаем функцию для возможности указания групп, по которым нужно проводить оценку выбросов на основе межквартильного размаха. И справним какое количество значений признается выбросом с учетом и без учета групп. Для минимального значения площади установим 6 квадратных метров, т.к. это минимальнае допустимае законодательством площадь для одного человека по санитарным нормам. Максимальное значение также будет ограничено 1.5 кратным межквартильным размахом от 3-го квартиля.

In [ ]:
# Добавление в функцию возможности группировки
def is_outlier(
    df: pd.DataFrame, target: str, by: list | None = None,
    min_val: int | None = None, max_val: int | None = None, 
    threshold=1.5, verbose=False
) -> pd.Series:
    df = df.copy()
    if by is None:
        if verbose: print("Выбросы без группировки")
        Q1 = df[target].quantile(0.25)
        Q3 = df[target].quantile(0.75)
        min_val = max(Q1 - threshold * (Q3 - Q1), min_val) if min_val else Q1 - threshold * (Q3 - Q1)
        if verbose: print("Минимальное значения для признания выбросом:", min_val)
        max_val = min(threshold * (Q3 - Q1) + Q3, max_val) if max_val else Q3 + threshold * (Q3 - Q1)
        if verbose: print("Максимальное значения для признания выбросом:", max_val)
    else:
        if verbose: print(f"Выбросы с группировкой по {by}")
        # Получение межквартильного отклонения для каждой группы
        vals = df.groupby(
            by=by)[target].quantile([.25, .75]).to_frame().unstack().droplevel(0, axis=1).apply(
            lambda x:
            pd.Series({
                "min_val": x[0.25] - threshold * (x[0.75] - x[0.25]),
                "max_val": x[0.75] + threshold * (x[0.75] - x[0.25])
            }), axis=1
        )
        # Добавление ограничений минимального и максимального значения
        if min_val: vals.loc[vals["min_val"] < min_val, "min_val"] = min_val
        if max_val: vals.loc[vals["max_val"] > max_val, "max_val"] = max_val
        if verbose:
            print("Значения для признания выбросом:")
            print(vals.T)
        # Добавление ограничений к каждой записи датасета by
        vals = df.join(vals, on=by)[["min_val", "max_val"]]
        min_val = vals["min_val"]
        max_val = vals["max_val"]
    return (df[target] < min_val) | (df[target] > max_val)

In [ ]:
print("Количество выбросов признака rooms (с группировкой):", 
      is_outlier(
          data, "total_area", by=["rooms", "building_type_int"], 
          min_val=6, verbose=True).sum())
print()
print("Количество выбросов признака rooms (без группировки):", 
      is_outlier(data, "total_area", min_val=6, verbose=True).sum())

В случае применения группировки по признакам, выбросами признаются 3819 записей, а без применения группировки признается практически в 2 раза больше записей. Произведем очистку от выбросов по признаку `total_area` с группировкой по `rooms` и `building_type_int`. Возможно более оптимальным решением будет увеличить коэффициент межквартильного размаха, однако такой вопрос лучше урегулировать с Заказчиком.

In [ ]:
data = data[~is_outlier(
    data, "total_area", by=["rooms", "building_type_int"], min_val=6)].copy()
print("Количество записей после удаления выбросов:", data.shape[0])

**Площадь кухни** имеет довольно большое количество нулевых значений в исходном датасете - порядка 8.3%, значения больше 0 начинаются с 1.5 квадратных метра и доходит до 203. Площадь кухни обычно заполняется более ответственно, однако ее необходимо рассматривать совместно с общей и жилой площадью.

**Жилая площадь** имеет еще больше нулевых значений в исходном датасете - 13.1%, значения более 0 начинаются с 2 квадратов и доходят до 700. Вероятнее всего нулевые значения просто не были заполнены пользователями. Проверим не выходят ли суммарные значения площади кухни и жилой площади за значения общей площади.

In [ ]:
display(data[(data["kitchen_area"] + data["living_area"]) > data["total_area"]])
print(f"Среднее отношение площади кухни к общей: {(data['kitchen_area'] / data['total_area']).mean():.2%}")
print(f"Среднее отношение жилой площади к общей: {(data['living_area'] / data['total_area']).mean():.2%}")

В измененном датасете имеется 139 записей, в которых суммарная площадь кухни и жилой превышает общую площадь. По таким данным можно отметить, что встречаются некорректные значения как площади кухни, так и жилой площади. Также некоторые значения очень близки и могут быть вызваны погрешностью типа float в Python. Однако жилая площадь рассчитывается не только за исключением кухни, но и прихожих, санузлов и прочих нежилых помещений. 

Лучше исправить некорректные значения на медиану по каждой группе в разрезе количества комнат и типа здания. Однако имея только площадь кухни или жилую невозможно определить насколько корректно она заполнена, необходимо оценить соотношение к общей площади. Дополним функцию поиска выбросов для обеспечения возможности определить их по соотношению к другому признаку.

In [ ]:
# Добавление возможности осуществлять оценку соотношения (разделить на другое поле)
# и исключение нулевых значений для рассчета выбросов
def is_outlier(
    df: pd.DataFrame, target: str, divider: str | None = None, out_val: int | None = None,
    by: list | None = None, min_val: int | None = None, max_val: int | None = None, 
    threshold = 1.5, verbose = False,
) -> pd.Series:
    df = df.copy()
    if divider is not None:
        df[target] = df[target] / df[divider]
    if by is None:
        if verbose: print("Выбросы без группировки")
        Q1 = df[target].quantile(.25) if out_val is None else df.query(f"{target} != {out_val}").quantile(.25)
        Q3 = df[target].quantile(.75) if out_val is None else df.query(f"{target} != {out_val}").quantile(.75)
        min_val = max(Q1 - threshold * (Q3 - Q1), min_val) if min_val else Q1 - threshold * (Q3 - Q1)
        if verbose: print("Минимальное значения для признания выбросом:", min_val)
        max_val = min(threshold * (Q3 - Q1) + Q3, max_val) if max_val else Q3 + threshold * (Q3 - Q1)
        if verbose: print("Максимальное значения для признания выбросом:", max_val)
    else:
        if verbose: print(f"Выбросы с группировкой по {by}")
        # Получение межквартильного отклонения для каждой группы
        vals = df.groupby(
            by=by)[target].quantile([.25, .75]).to_frame().unstack().droplevel(0, axis=1).apply(
                lambda x:
                pd.Series({
                    "min_val": x[0.25] - threshold * (x[0.75] - x[0.25]),
                    "max_val": x[0.75] + threshold * (x[0.75] - x[0.25])
                }), axis=1
        ) if out_val is None else df.query(f"{target} != {out_val}").groupby(
            by=by)[target].quantile([.25, .75]).to_frame().unstack().droplevel(0, axis=1).apply(
                lambda x:
                pd.Series({
                    "min_val": x[0.25] - threshold * (x[0.75] - x[0.25]),
                    "max_val": x[0.75] + threshold * (x[0.75] - x[0.25])
                }), axis=1
        )
        # Добавление ограничений минимального и максимального значения
        if min_val is not None: vals.loc[vals["min_val"] < min_val, "min_val"] = min_val
        if max_val is not None: vals.loc[vals["max_val"] > max_val, "max_val"] = max_val
        if verbose:
            print("Значения для признания выбросом:")
            print(vals.T)
        # Определение ограничений каждой записи датасета
        vals = df.join(vals, on=by)[["min_val", "max_val"]]
        min_val = vals["min_val"]
        max_val = vals["max_val"]
    return (df[target] < min_val) | (df[target] > max_val) | (df[target] == out_val)

 Оценим количество выбросов в оставшемся датасете.

In [ ]:
kitchen_outs = is_outlier(
    data, "kitchen_area", divider="total_area", 
    out_val=0, by=["rooms", "building_type_int"],
)

living_outs = is_outlier(
    data, "living_area", divider="total_area",
    out_val=0, by=["rooms", "building_type_int"]
)

print(f"Некорректные значения площади кухни: {kitchen_outs.mean():.2%}")
print(f"Некорректные значения жилой площади: {living_outs.mean():.2%}")
print(f"Всего уникальных некорректных записей: {(kitchen_outs | living_outs).mean():.2%}")

Практически 19.5% записей вероятно заполнены некорректно. Заполним такие данные медианой по группам.

In [ ]:
# Функция для рассчета медианы
def get_agg_values(
    data: pd.DataFrame, target: str, divider: str | None = None,
    by: list | None = None, round_n = 2, func="median"
) -> pd.Series:
    result = data[target].copy()
    if by is None:
        result = result.transform(func=func)
    else:
        if divider is not None:
            result = result / data[divider]
        result = data[by].join(result.rename("result"))
        result = result.groupby(by).transform(func=func)["result"]
        if divider is not None:
            result = round(result * data[divider], round_n)
    return result

In [ ]:
# Исключение некорректных значений для рассчета медианы
data.loc[kitchen_outs, "kitchen_area"] = np.nan
data.loc[living_outs, "living_area"] = np.nan

# Получение медианных значений по датасету 
kitchen_median = get_agg_values(
    data, "kitchen_area", divider="total_area", by=["rooms", "building_type_int"]
)
living_median = get_agg_values(
    data, "living_area", divider="total_area", by=["rooms", "building_type_int"]
)

# Запись медианных значений в датасет
data.loc[kitchen_outs, "kitchen_area"] = kitchen_median[kitchen_outs]
print("По признаку kitchen_area произведена замена записей:",
      (data["kitchen_area"] == kitchen_median).sum())
data.loc[living_outs, "living_area"] = living_median[living_outs]
print("По признаку living_area произведена замена записей:",
      (data["living_area"] == living_median).sum())

Произведено исправление вероятно некорректно заполненных значений. Можно отметить, что фактические значения площади кухни и жилой площади не несут большой информации для модели, лучше было бы рассмотреть их отношение к общей площади.

### Этаж

По признаку этажа наименьшее значение - 1. Необходимо проверить, не превышает ли этаж квартиры максимальное количество этажей в здании.

In [ ]:
print("Количество квартир, этаж которых выше этажности здания:",
      (data["floor"] > data["floors_total"]).sum())

В датасете отсутствуют квартиры, этаж которых превышает этажность здания, возможно осуществляется такая проверка при создании объявлений.

Вероятно полезным было бы выделить квартиры, находящиеся на первом и последнем этажах. Оценим среднюю стоимость квадратного метра для таких квартир по сравнению с остальными.

In [ ]:
fig = px.histogram(
    x=data["price"] / data["total_area"],
    y=data.apply(
        lambda x: "Последний" if x["floor"] == x["floors_total"] else 
        "Первый" if x["floor"] == 1 else "Другой",
        axis=1
    ), category_orders={"y": ["Первый", "Последний", "Другой"]},
    title="Средняя цена квадратного метра по этажу",
    labels={"y": "Этаж"}, histfunc="avg"
)
fig.update_layout(xaxis_title="Средняя цена за кв. метр")
fig.show(renderer="png", width=1400)

Средняя цена за квадратный метр первого этажа значительно ниже средней цены других этажей, при этом последний этаж влияет на цену не так существенно.

Стоит отметить наличие в датасете не только этажности здания, но и количества квартир `floors_total` в здании. По таким данным можно приблизительно рассчитать плотность квартир на этаж, что может дополнительно подсказать оценочный размер здания - "муравейник" или "башня". Если бы датасет содержал также количество подъездов, можно было бы более точно определить плотность квартир. 

Дополнительно можно оценить влияние средней цены за квадратный метр этажа квартиры в домах без лифта.

In [ ]:
fig = px.histogram(
    x=data["floor"], y=data["price"] / data["total_area"],
    color=data["has_elevator"], barmode="group",
    title="Средняя цена квадратного метра по этажу и наличию лифта",
    labels={"x": "Этаж", "color": "Лифт"}, histfunc="avg"
)
fig.update_layout(yaxis_title="Средняя цена за кв. метр")
fig.show(renderer="png", width=1400)

Вероятно в датасете имеются ошибки по признаку наличия лифта. Сложно представить дома с квартирами на 40-м этаже в здании без лифта. При этом сильно выделяется средняя стоимость квадратного метра выше 5 этажа. Квартиры до 5 этажа включительно вероятнее всего относятся к большой массе 5-этажных домов, где обычно лифт действительно отсутствует. Квартиры выше 5-го этажа без лифта существенно отличаются средней ценой от остальной массы квартир.

### Студии и Апартаменты

Признак `studio` представлен только значением `False`, вероятно доступные нам данные отфильтрованы по этому признаку, однако сам признак остался. Исключим данный столбец. 

In [ ]:
data.drop(columns="studio", inplace=True)
print("Новый размер датасета:", data.shape)

По апартаментам можно отметить наличие слабой корреляции с признаком типа здания. Оценим к каким типам здания относятся апартаменты.

In [ ]:
fig = px.histogram(
    data.query("is_apartment == True"), x="building_type_int",
    title="Распределение апартаментов по типам зданий",
    labels={"building_type_int": "Тип здания", "count": "количество"}
)
fig.update_layout(bargap=0.2)
fig.show(renderer="png", width=1400)

Основная масса апартаментов располагается в зданиях типа `2`, при этом имеется немалое количество в зданиях типа `4` и `1`, в зданиях других типов количество апартаментов минимально, а в зданиях типа `5` отсутствуют. Вероятно, в виду отсутствия апартаментов в зданиях типа `5` данные эти признаки показывают наличие корреляции.

### Тип здания

In [ ]:
fig = px.histogram(
    data, x="building_type_int", histfunc="count",
    title="Распределение типов зданий в выборке",
    labels={"building_type_int": "Тип здания"}
)
fig.update_layout(yaxis_title="Количество", bargap=0.2)
fig.show(renderer="png", width=1400)

Основная масса зданий приходится на 4-й тип, достаточно большое количество зданий приходится на 2-й и 1-й типы зданий. Немного ниже количество зданий типа 6. Наименьшее количестов зданий приходится на типы 3 и 0, а меньше всего 5.

In [ ]:
fig = px.histogram(
    data, x="building_type_int", y="price", histfunc="avg", 
    title="Средние значения цены по типу здания",
    labels={"count": "Средняя цена", "building_type_int": "Тип здания"}
)
fig.update_layout(yaxis_title="Средняя цена", bargap=0.2)
fig.show(renderer="png", width=1400)

Наибольшая средняя цена приходится на 3 тип зданий, остальные в среднем существенно дешевле. Данный признак явно является категориальным, причем признак имеет нелинейное отношение к целевому. Для более точной оценки необходимо сравнить с ценой квадратного метра.

In [ ]:
fig = px.histogram(
    x=data["building_type_int"], y=data["price"] / data["total_area"],
    title="Средняя цена за квадрат по типу здания",
    labels={"x": "Тип здания"}, histfunc="avg"
)
fig.update_layout(yaxis_title="Средняя цена", bargap=0.2)
fig.show(renderer="png", width=1400)

Значения средней цены за квадратный метр существенно ближе. Однако также наблюдается отсутствие прямой или обратной зависимости значений с целевым признаком. Данный признак явно является категориальным и не имеет порядковой составляющей, по крайне мере по отношению к целевому признаку, соответственно лучше его преобразовать в текст, чтобы в дальнейшем его было легче определить как категориальный.

In [ ]:
data["building_type_int"] = data["building_type_int"].astype("str")
data.rename(columns={"building_type_int": "building_type"}, inplace=True)
data["building_type"].unique()

### Картография

Данные по широте и долготе распределены в пределах одного градуса и в целом не выглядят аномально. Оценим расположение случайных 5000 точек на карте.

In [ ]:
fig = px.scatter_mapbox(
    data_frame=data.sample(5000, random_state=RANDOM_STATE), 
    lat="latitude", lon="longitude", hover_name="rooms",
    mapbox_style="open-street-map", zoom=8)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Судя по карте датасет состоит из квартир города Москва. Преобразование данных не требуется.

Вероятно полезным было бы добавление в датасет информации о расстоянии до ближайших социально значимых объектов, таких как станции метро, школы, детские сады, парки и другие, а также расстояние до центра города.

### Высота потолков

Высота потолков начинается с 2 метров и доходит до максимального значения в 27, второе максимальное значение - 8 метров. В целом в сервисе Яндекс Недвижимость можно встретить двухуровневые и более квартиры со вторым светом, высота потолков которых может доходить до 8 метров или даже выше, однако значение в 27 метров возможно является ошибкой. Для обучения модели лучше ограничим значение высоты потолков от 2-х до 10-ти метров.

In [ ]:
data = data.query("2 <= ceiling_height <= 10")
print("Новый размер датасета:", data.shape)

### Другие признаки дома

По остальным признакам, относящимся к характеристикам дома аномалии не наблюдаются. Признаки останутся без преобразования.

## Анализ целевой переменной

По целевому признаку в исходном датасете были большие выбросы. Возможно от части выбросов удалось избавиться в предыдущих этапах. Оценим распределение целевого признака после произведенных преобразований.

In [ ]:
fig = px.box(data, x="price", title="Boxplot для целевого признака", labels={"price": "Цена"})
fig.show(renderer="png", width=1400)

Очень большой разброс по цене, основные значения существенно ниже чем максимальные. Взглянем на данные с ограничением по максимальному значению.

In [ ]:
fig = px.box(data, x="price", title="Boxplot для целевого признака (до 100'000'000)", labels={"price": "Цена"})
fig.update_xaxes(range=[0, 1e8])
fig.show(renderer="png", width=1400)

Разброс целевого признака очень большой, основная масса сотредоточена в нижней части значений в пределах до 30 000 000, однако рассматривать целевой признак как есть не имеет смысла, нужно оценить разброс значений не по общей стоимости квартиры, а по стоимости квадратного метра.

In [ ]:
fig = px.box(
    x=data["price"] / data["total_area"], 
    title="Boxplot для цены за квадратный метр",
    labels={"x": "Цена за кв. метр"}
)
fig.show(renderer="png", width=1400)

По цене за квадратный метр наблюдается близкая картина - имеются сильные выбросы. Оценим основную массу значений.

In [ ]:
fig = px.box(
    x=data["price"] / data["total_area"], 
    title="Boxplot для цены за квадратный метр (до 500'000)",
    labels={"x": "Цена за кв. метр"}
)
fig.update_xaxes(range=[0, 500000])
fig.show(renderer="png", width=1400)

### Анализ целевой переменной в зависимости от различных признаков

По целевому признаку наибольшая корреляция наблюдается с признаками площади `total_area` (0.76), `living_area` (0.73) и `kitchen_area` (0.55), другими числовыми признаками `rooms` (0.62) и `ceiling_height` (0.44) , а также с категориальной `building_type_int` (0.48). Оценим признаки датасета.

Взглянем на взаимосвязь целевого признака с наиболее коррелирующим признаком общей площади, а также в разрезе наиболее коррелирующего категориального признака типа дома. 

In [ ]:
fig = px.scatter(
    data, x="price", y="total_area", color="building_type",
    height=600, title="Диаграмма рассеяния цен квартир по площади в разрезе типов зданий",
    labels={"price": "Цена", "total_area": "Общая площадь", "building_type": "Тип здания"},
    category_orders={"building_type":[str(x) for x in range(7)]}
)
fig.show(renderer="png", width=1400)

В связи с очень большими выбросами ограничим вывод цены до 100 000 000.

In [ ]:
fig = px.scatter(
    data, x="price", y="total_area", color="building_type",
    height=600, title="Диаграмма рассеяния цен квартир по площади в разрезе типов зданий (до 100'000'000)",
    labels={"price": "Цена", "total_area": "Общая площадь", "building_type": "Тип здания"},
    category_orders={"building_type":[str(x) for x in range(7)]}
)
fig.update_xaxes(range=[0, 1e8])
fig.show()

Можно отметить, что некоторые цены являются довольно популярными (вертикальные полосы из точек), куда попадают квартиры с различной общей площадью. Зависимость цены квартиры от площади нелинейна, причем имеются довольно большой разброс значений по цене. Судя по всему существенное влияние на такой большой разброс цен может также оказывать расположение квартир в различных районах города.

In [ ]:
# Определение глубины округления широты/долготы
round_scale = 2
geo_step = 10 ** -round_scale / 2

# Получение аггрегирующих данных
map_data = pd.DataFrame({
    "lat": data["latitude"].round(round_scale), 
    "lon": data["longitude"].round(round_scale), 
    "price": data["price"] / data["total_area"]
}).groupby(by=["lat", "lon"])["price"].agg(["min", "median", "max"]).reset_index().reset_index()

# Получение сетки для вывода на карте
geojson = {
    "type": "FeatureCollection",
    "features": map_data.apply(
        lambda x:
        {   
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [x["lon"]-geo_step, x["lat"]-geo_step],
                    [x["lon"]+geo_step, x["lat"]-geo_step],
                    [x["lon"]+geo_step, x["lat"]+geo_step],
                    [x["lon"]-geo_step, x["lat"]+geo_step],
                    [x["lon"]-geo_step, x["lat"]-geo_step]
                ]]
            },
            "properties": {
                "id": x["index"],
                "min": x["min"],
                "med": x["median"],
                "max": x["max"]
            }
        }, axis=1
    ).to_list()
}

# Вывод тепловой карты по регионам
fig = px.choropleth_mapbox(
    map_data, geojson=geojson, color="median",
    locations="index", featureidkey="properties.id",
    color_continuous_scale="Bluered",
    mapbox_style="open-street-map", zoom=8, opacity=0.8,
    center={"lat": map_data["lat"].mean(), "lon": map_data["lon"].mean()}
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

По средней цене за квадратный метр можно отметить довольно высокие цены вблизи центра Москвы, а также местами в западной части города. Обработку выбросов для целевого признака некорректно проводить без привязки к местоположению квартиры, так как разные местоположения достаточно сильно влияют на целевой признак. 

Добавим группировку по широте и долготе, однако для этого необходимо округлить значения до 2-го разряда дробной части, так как дробные разряды более 2 дают слишком маленький размер региона. Доработаем функцию определения выбросов, в том числе с учетом обработки без использования межквартильного размаха (для высоты потолка).


In [ ]:
# Добавление округления значений столбцов для группировки и статичных ограничений (без межквартильного размаха)
def is_outlier(
    df: pd.DataFrame, target: str, divider: str | None = None,  out_val: int | None = None,
    by: list | None = None, by_round: dict | None = None, min_val: int | None = None, 
    max_val: int | None = None, threshold: float | None = 1.5, verbose = False,
) -> pd.Series:
    df = df.copy()
    if divider is not None:
        df[target] = df[target] / df[divider]
    if by is None:
        if verbose: print("Выбросы без группировки")
        if threshold is not None:
            Q1 = df[target].quantile(.25) if out_val is None else df.query(f"{target} != {out_val}").quantile(.25)
            Q3 = df[target].quantile(.75) if out_val is None else df.query(f"{target} != {out_val}").quantile(.75)
            min_val = max(Q1 - threshold * (Q3 - Q1), min_val) if min_val else Q1 - threshold * (Q3 - Q1)
            max_val = min(threshold * (Q3 - Q1) + Q3, max_val) if max_val else Q3 + threshold * (Q3 - Q1)
        else:
            if (min_val is None) or (max_val is None):
                raise ValueError("Если не используется аргумент 'threshold' необходимо обязательно указать 'min_value' и 'max_value'")
        if verbose: print("Минимальное значения для признания выбросом:", min_val)
        if verbose: print("Максимальное значения для признания выбросом:", max_val)
    else:
        if verbose: print(f"Выбросы с группировкой по {by}")
        # Округляем значения столбцов
        if by_round is not None:
            for col, val in by_round.items():
                df[col] = df[col].round(val)
        # Получение межквартильного отклонения для каждой группы
        vals = df.groupby(
            by=by)[target].quantile([.25, .75]).to_frame().unstack().droplevel(0, axis=1).apply(
                lambda x:
                pd.Series({
                    "min_val": x[0.25] - threshold * (x[0.75] - x[0.25]),
                    "max_val": x[0.75] + threshold * (x[0.75] - x[0.25])
                }), axis=1
        ) if out_val is None else df.query(f"{target} != {out_val}").groupby(
            by=by)[target].quantile([.25, .75]).to_frame().unstack().droplevel(0, axis=1).apply(
                lambda x:
                pd.Series({
                    "min_val": x[0.25] - threshold * (x[0.75] - x[0.25]),
                    "max_val": x[0.75] + threshold * (x[0.75] - x[0.25])
                }), axis=1
        )
        # Добавление ограничений минимального и максимального значения
        if min_val is not None: vals.loc[vals["min_val"] < min_val, "min_val"] = min_val
        if max_val is not None: vals.loc[vals["max_val"] > max_val, "max_val"] = max_val
        if verbose:
            print("Значения для признания выбросом:")
            print(vals.T)
        # Определение ограничений каждой записи датасета
        vals = df.join(vals, on=by)[["min_val", "max_val"]]
        min_val = vals["min_val"]
        max_val = vals["max_val"]
    return (df[target] < min_val) | (df[target] > max_val) | (df[target] == out_val)


Оценим количество вероятных выбросов по целевому признаку. Для этих целей будем также использовать аггрегацию по признакам - количество комнат, у которого высокая корреляция с общей площадью и его можно использовать как категориальный, а также признак типа здания. Чтобы учесть географическую составляющую также добавим в группировку округленные значения широты и долготы до 2-го разряда дробной части.

In [ ]:
print(
    "Количество вероятных выбросов по целевому признаку:",
    is_outlier(
        data, target="price", by=["rooms", "building_type", "latitude", "longitude"], 
        by_round={"latitude": 2, "longitude": 2}, divider="total_area", min_val=0
    ).sum()
)

Количество таких выбросов не большое по сравнению с размером датасета, считаю целесообразным очистить данные от выбросов по целевому признаку.

In [ ]:
data = data[~is_outlier(
    data, target="price", by=["rooms", "building_type", "latitude", "longitude"], 
    by_round={"latitude": 2, "longitude": 2}, divider="total_area", verbose=False
)].copy()
print("Новый размер датасета:", data.shape)

## Обзор датасета после обработки выбросов

Для проведения предобработки выбросов подготовим sklearn трансформер, который выполнит полную предобработку "сырых" данных.

In [ ]:
# Трансформер для проведения предобработки выбросов
class OutlierProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, outlier_params=None, dropna=True, drop_duplicates=True, verbose=False):
        self.outlier_params = outlier_params
        self.dropna = dropna
        self.drop_duplicates=drop_duplicates
        self.verbose = verbose
    
    # Добавление ранее объявленных методов как статические
    is_outlier = staticmethod(is_outlier)
    get_agg_values = staticmethod(get_agg_values)

    # Метод fit не производит никаких вычислений
    def fit(self, X, y=None):
        # Хорошей практикой было бы добавить в этот метод поиск и сохранение всех границ значений,
        # а в методе transform проводить сравнение и обработку значений для определения выбросов,
        # но это бы сильно поменяло логику ранее подготовленной мной функции для DAG.
        # Поэтому не стал сильно переделывать, как обычно отстаю.
        return self
    
    # Метод transform производит обработку выбросов
    def transform(self, X, y=None):
        # Работа с копией полученных данных
        result = X.copy()
        if self.verbose: print("Размер датасета:", result.shape)
        # Обработка признаков
        if self.outlier_params is not None:
            # Удаление неиспользуемых признаков
            if "drop_cols" in self.outlier_params:
                result.drop(columns=self.outlier_params["drop_cols"], inplace=True)
                if self.verbose: 
                    print("Удалены признаки", self.outlier_params["drop_cols"])
            # Удаление дубликатов перед обработкой выбросов
            if self.drop_duplicates: 
                if self.verbose: print("Количество дубликатов:", result.duplicated().sum())
                result.drop_duplicates(keep="first", inplace=True)
                if self.verbose: print("Размер датасета после удаления дубликатов:", result.shape)
            # Обработка выбросов
            if "out_cols" in self.outlier_params:
                for col in self.outlier_params["out_cols"]:
                    if self.verbose: print("Обработка признака", col["name"])
                    result.loc[self.is_outlier(
                        result, target=col["name"], 
                        divider=col["divider"] if "divider" in col else None,
                        out_val=col["out_val"] if "out_val" in col else None,
                        by=col["by"] if "by" in col else None, 
                        by_round=col["by_round"] if "by_round" in col else None,
                        min_val=col["min_val"] if "min_val" in col else None,
                        max_val=col["max_val"] if "max_val" in col else None,
                        threshold=col["threshold"] if "threshold" in col else None,
                        verbose=self.verbose
                    ), col["name"]] = np.nan
                    if self.verbose: 
                        print(f"Исключены выбросы в признаке {col['name']}: {result[col['name']].isna().sum()}")
            # Заполнение пропусков медианой
            if "fill_cols" in self.outlier_params:
                for col in self.outlier_params["fill_cols"]:
                    if self.verbose: print("Заполнение пропусков признака", col["name"])
                    outs = result[col["name"]].isna()
                    if self.verbose: print("Количество пропусков:", outs.sum())
                    result.loc[outs, col["name"]] = self.get_agg_values(
                        result, target=col["name"],
                        divider=col["divider"] if "divider" in col else None,
                        by=col["by"] if "by" in col else None,
                        round_n=col["round"] if "round" in col else None,
                        func=col["func"] if "func" in col else "median",
                    )[outs]
                    if self.verbose: print("Пропусков осталось:", result[col["name"]].isna().sum())
            if "change_col" in self.outlier_params:
                for col in self.outlier_params["change_col"]:
                    if "astype" in col:
                        result[col["name"]] = result[col["name"]].astype(col["astype"])
                    if "rename" in col:
                        result.rename(columns={col["name"]: col["rename"]}, inplace=True)
        # Удаление пропусков, в том числе выбросов, которым присвоено np.nan
        if self.dropna:
            if self.verbose: 
                print("Количесво пропусков:")
                print(result.isna().sum())
            result.dropna(inplace=True)
            if self.verbose: print("Размер датасета после удаления пропусков:", result.shape)

        # Удаление дубликатов после обработки выбросов
        if self.drop_duplicates: 
            if self.verbose: print("Количество дубликатов:", result.duplicated().sum())
            result.drop_duplicates(keep="first", inplace=True)
            if self.verbose: print("Размер датасета после удаления дубликатов:", result.shape)
        # Возврат результатов
        return result

Настройки для трансформера для обработки выбросов располагаются в файле `outlier_params.yaml`:

In [ ]:
# Параметры для обработки выброса для ноутбука с EDA
outlier_params = {
    'drop_cols': ['id', 'studio'], 
    'out_cols': [
        {
            'name': 'rooms', 
            'min_val': 1, 
            'threshold': 1.5
        }, 
        {
            'name': 'total_area', 
            'by': ['rooms', 'building_type_int'], 
            'min_val': 6, 
            'threshold': 1.5
        }, 
        {
            'name': 'kitchen_area', 
            'divider': 'total_area', 
            'out_val': 0, 
            'by': ['rooms', 'building_type_int'], 
            'threshold': 1.5}, 
        {
            'name': 'living_area', 
            'divider': 'total_area', 
            'out_val': 0, 
            'by': ['rooms', 'building_type_int'], 
            'threshold': 1.5
        }, 
        {
            'name': 'ceiling_height', 
            'min_val': 2, 
            'max_val': 10
        }, 
        {
            'name': 'price', 
            'divider': 'total_area', 
            'by': ['rooms', 'building_type_int', 'latitude', 'longitude'], 
            'by_round': {'latitude': 2, 'longitude': 2}, 
            'min_val': 0, 'threshold': 1.5
        }
    ], 
    'fill_cols': [
        {
            'name': 'kitchen_area', 
            'divider': 'total_area', 
            'by': ['rooms', 'building_type_int'], 
            'round': 2, 
            'func': 'median'
        }, 
        {
            'name': 'living_area', 
            'divider': 'total_area', 
            'by': ['rooms', 'building_type_int'], 
            'round': 2, 
            'func': 'median'
        }
    ], 
    'change_col': [
        {
            'name': 'building_type_int', 
            'astype': 'str', 
            'rename': 'building_type'
        }
    ]
}

Проведем предобработку исходных данных и справним два датасета - подготовленный вручную в рамках EDA и полученный в результате предобработки трансформером.

In [ ]:
report_floor = sv.compare(
    (data.drop(columns="id").drop_duplicates(keep="first"), "Вручную"),
    (
        OutlierProcessor(
            outlier_params=outlier_params,
            #verbose=True
        ).fit_transform(get_data()), 
        "Трансформер"
    )
)
report_floor.show_notebook()

На первый взгляд распределение в датасетах выглядит идентично, однако можно обнаружить небольшие расхождения по значениям площади кухни и жилой, которые были заполнены медианой. Это связано с тем, что расчет медианы для площадей при обработке вручную производился до исключения выбросов по оставшимся признакам, а в трансформере производится после, чтоя является более корректным.

In [ ]:
fig = px.scatter_matrix(
    data, dimensions=[
        "total_area", "kitchen_area", "living_area", "floor", "ceiling_height", "price"
    ], color="building_type", width=1000, height=900,
    title="Матрица рассеяния признаков квартиры",
    labels={"building_type": "Тип дома"},
    category_orders={"building_type":[str(x) for x in range(7)]}
)
fig.show(renderer="png")

По графику заметно, что наиболее высокие цены приходятся на квартиры большей площади, которые располагаются в домах типа 3 и 2. Другие типы домов существенно дешевле. Имеется довольно сильная линейная зависимость жилой площади от общей. По площади кухни имеется больший разброс относительно общей площади. 

Оценим признаки домов, в котором располагаются квартиры.

In [ ]:
fig = px.scatter_matrix(
    data, dimensions=[
        "build_year", "latitude", "longitude", "flats_count", "floors_total", "price"
    ], color="building_type", width=1000, height=900,
    title="Матрица рассеяния признаков дома",
    labels={"building_type": "Тип дома"},
    category_orders={"building_type":[str(x) for x in range(7)]}
)
fig.show(renderer="png")

По признакам дома можно отметить, что очень сильное влияние на цену квартиры оказывает местоположение дома - наибольшие цены располагаются вблизи центра города. Хорошо было бы добавить в датасет признак близости к центру города.

## Выводы после EDA

По результатам проведенного исследовательского анализа данных произведено следующее:
1. Исключены дубликаты в данных;
2. Выявлены и исключены выбросы и редкие значения по следующим признакам:
    - количество комнат `rooms` за пределами межквартильного размаха в 1.5 IQR от Q1 и Q3; 
    - общая площадь `total_area` аналогичным статистическим методом в разрезе количества комнат и типов зданий;
    - целевой признак цены квартиры `price` исходя из цены за квадратный метр в разрезе количества комнат, типов зданий и местоположения объекта методом межквартильного размаха;
    - высота потолков `ceiling_height` путем исключения значений за пределами промежутка от 2-х до 10 метров;
    - в случае наличия пропусков в данных они исключаются из датасета.
3. Проведена обработка вероятно некорректных значений признаков площади кухни `kitchen_area` и жилой площади `living_area`, которая производилась также методом межквартильного размаха, признанные выбросом значения установлены медианой по соотношению к общей площади в разрезе количества комнат и типов зданий.
4. Признак `building_type_int` не является  переименован в `building_type` и преобразован в категориальный.

Для автоматической обработки выбросов подготовлен трансформер и файл для параметров его работы. По результатам обработки признаков можно отметить следующее:
1. В части корреляции целевого признака с остальными:
    - наибольшая корреляция с признаком общей площади `total_area` (0.75), однако зависимость нелинейная и имеет довольно большой разброс;
    - также достаточно высокая корреляция по другим признакам площади `living_area` (0.73) и `kitchen_area` (0.56), однако данные признаким имеют достаточно большую зависимость друг от друга и признака `total_area`;
    - заметная корреляция с количеством комнат `rooms` (0.59), зависимость также нелинейная с большим разбросом;
    - умеренная корреляция с типом здания `building_type` (0.45);
    - также умеренная корреляция с высотой потолков `ceiling_height` (0.45); 
    - слабая корреляция с годом постройки `build_year` (0.11), по остальным признакам корреляция практически отсутствует;
    - средние цены квадратного метра в центральной части Москвы выше и снижаются по мере удаления.
2. В части мультиколлинеарности датасета:
    - очень высокая корреляция между признаками `total_area` и `living_area` - 0.97, что явно говорит о мультиколлинеарности;
    - довольно высокая корреляция признака `rooms` с признаками `living_area` (0.88) и `total_area` (0.85);
    - по признаку `building_type` имеется корреляция с большинством признаков датасета, в частности с `build_year` корреляция составляет 0.72, по остальным признакам ниже.

На основании проведенного EDA в части подготовки дополнительных признаков можно отметить следующее:
- признаки количества комнат и площади имеют нелинейное отношение к целевому, вероятно наличие степенной зависимости. Цена существенно повышается по мере увеличения значения этих признаков.
- возможно более оптимальным вариантом было бы использование не фактического значения признаков площади кухни `kitchen_area` и жилой площади `living_area`, а относительное к общей площади `total_area`;
- для некоторых моделей могут быть полезны признаки нахождения на первом и последнем этажах, однако для моделей, основанных на деревьях решений выделение такого признака не имеет большого значения;
- признак количества этажей в зании можно преобразовать в классы по высоте - небольшой этажности, более высокие и высотки;
- также вероятно полезным может быть плотность квартир на этаж - `flats_count` / `floors_total` или другое их соотношение. Возможно более оптимальным была бы плотность квартир на этаж в подъезде, однако в датасете отсутствуют сведения о количестве подъездов в доме;
- полезными могут быть сведения о наличии лифта на высоких этажах, однако, вероятнее всего, в датасете имеются ошибки в таких сведениях. При этом можно отметить различие средней цены для квартир до 5-го этажа включительно и квартир выше 5-го этажа, в зданиях без лифта;
- в части картографических данных было бы полезно иметь сведения о расстоянии до до центра города , а также до наиболее социально значимых объектов - станции метро, школы, парки и т.д. Однако такие сведения отсутствуют в датасете;
- вероятно имеются и другие сложные или скрытые зависимости между признаками, которые могут быть сгенерированы автоматически.

При проведении исследовательского анализа были определены следующие категориальные признаки - тип здания `building_type`, апартаменты `is_apartment`, наличие лифта `has_elevator`. Признак  студии `studio` был исключен, так как подобные значения отсутствуют в выборке и обучать модель по такому признаку не имеет смысла. Из категориальных признаков только `building_type` не является бинарным. Остальные признаки являются числовыми.


В результате подготовки данных были исключены выбросы значений признаков, поэтому наиболее предпочтительной метрикой является RMSE, результаты можно дополнительно оценивать метрикой MAPE, чтобы понимать расхождение в процентном соотношении. Несмотря на то, что в качестве модели можно выбрать простые модели машинного обучения, в данном проекте планируется применение на ансамблевых методов, наиболее точными из которых являются бустинговые модели. Из бустинговых моделей наибольшую популярность имеют XGBoost, LightGBM и CatBoost. Несмотря на то, что категориальных признаков немного и данные подготовлены для работы любых моделей, выбрана модель CatBoost так как она обладает высокой точностью и простотой настройки параметров. При этом данных не так много, чтобы делать выбор в пользу более быстрых моделей бустинга.

- Метрики: **RMSE**, **MAPE**.
- Модель: **CatBoostRegressor**

Разделение датасета будет производиться на тренировочную и тестовую выборки, с проведением кросс-валидации на тренировочной выборке для определения наилучших параметров для модели. Определены бинарные, категориальные и числовые признаки, а также целевой признак. Бинарные признаки будут закодированы посредством OneHotEncoder. Так как единственный категориальный признак не является порядковым, а количество значений небольшое, его можно кодировать OneHotEncoder и это будет наилучшим выбором для большинства моделей, но в данном проекте решено закодировать его посредством CatBoostEncoder. Для некоторых простых моделей желательно закодировать числовые признаки, чтобы привести их к единому масштабу, однако в данном проекте предполагается использование более продвинутых ансамблевых методов, основанных на деревьях решений, поэтому кодирование числовых признаков не имеет смысла.